<a href="https://colab.research.google.com/github/HegdeChaitra/speech/blob/master/weather2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np


import pandas as pd
import matplotlib.pyplot as plt
import sklearn

import numpy as np
import pandas as pd
import math
import sklearn
import sklearn.preprocessing
import datetime
import os
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, models
from torchvision import transforms as T
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from sklearn.utils.class_weight import compute_class_weight

In [0]:
df = pd.read_csv("jfk_weather_cleaned.csv")

In [3]:
df.head()

,DATE,VISIBILITY,DRYBULBTEMPF,WETBULBTEMPF,DewPointTempF,RelativeHumidity,WindSpeed,WindDirection,StationPressure,SeaLevelPressure,Precip
0,2010-01-01 00:51:00,6.0,33,32,31,92,0,0,29.97,29.99,0.01
1,2010-01-01 01:51:00,6.0,33,33,32,96,0,0,29.97,29.99,0.02
2,2010-01-01 02:51:00,5.0,33,33,32,96,0,0,29.97,29.99,0.02
3,2010-01-01 03:51:00,5.0,33,33,32,96,0,0,29.95,29.97,0.02
4,2010-01-01 04:51:00,5.0,33,32,31,92,0,0,29.93,29.96,0.02


In [4]:
df.columns

Index(['DATE', 'VISIBILITY', 'DRYBULBTEMPF', 'WETBULBTEMPF', 'DewPointTempF',
       'RelativeHumidity', 'WindSpeed', 'WindDirection', 'StationPressure',
       'SeaLevelPressure', 'Precip'],
      dtype='object')

In [0]:
df['DATE']=pd.to_datetime(df['DATE'])

In [0]:
df['month'] = df['DATE'].dt.month

In [7]:
df.head()

,DATE,VISIBILITY,DRYBULBTEMPF,WETBULBTEMPF,DewPointTempF,RelativeHumidity,WindSpeed,WindDirection,StationPressure,SeaLevelPressure,Precip,month
0,2010-01-01 00:51:00,6.0,33,32,31,92,0,0,29.97,29.99,0.01,1
1,2010-01-01 01:51:00,6.0,33,33,32,96,0,0,29.97,29.99,0.02,1
2,2010-01-01 02:51:00,5.0,33,33,32,96,0,0,29.97,29.99,0.02,1
3,2010-01-01 03:51:00,5.0,33,33,32,96,0,0,29.95,29.97,0.02,1
4,2010-01-01 04:51:00,5.0,33,32,31,92,0,0,29.93,29.96,0.02,1


In [0]:
df = df[['DATE', 'DRYBULBTEMPF', 'WETBULBTEMPF', 'DewPointTempF','VISIBILITY', 
       'RelativeHumidity', 'WindSpeed', 'WindDirection', 'StationPressure',
       'SeaLevelPressure', 'Precip','month']]

In [9]:
df.shape

(75083, 12)

In [0]:
def load_data(stock, y, seq_len):
    data_raw = np.array(stock) # convert to numpy array
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - seq_len): 
        data.append(data_raw[index: index + seq_len])
    
    data = np.array(data);
    target_out = y[seq_len:]
    
    print(data.shape, len(target_out))
    valid_set_size = int(np.round(valid_set_size_percentage/100*data.shape[0]));  
    test_set_size = int(np.round(test_set_size_percentage/100*data.shape[0]));
    train_set_size = data.shape[0] - (valid_set_size + test_set_size);
    
    x_train = data[:train_set_size,:,:]
    y_train = target_out[:train_set_size]
    
    x_valid = data[train_set_size:train_set_size+valid_set_size,:,:]
    y_valid = target_out[train_set_size:train_set_size+valid_set_size]
    
    x_test = data[train_set_size+valid_set_size:,:,:]
    y_test = target_out[train_set_size+valid_set_size:]
    
    return [x_train, y_train, x_valid, y_valid, x_test, y_test]

In [0]:
valid_set_size_percentage = 20
test_set_size_percentage = 20

In [12]:
out = load_data(df[['DATE', 'VISIBILITY', 'RelativeHumidity', 'WindSpeed', 'WindDirection', 'StationPressure',
       'SeaLevelPressure', 'Precip','month']], list(df['DRYBULBTEMPF']),20)

(75063, 20, 9) 75063


In [0]:
class BrainImages(Dataset):
    def __init__(self, x, y):
        
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self,idx):
        a = [[float(x) for x in a] for a in self.x[idx][:,1:]]
        # print(np.array(self.x[idx][:,1:]))
        # print(a)
        xi = np.array(a)
        yi = np.array([self.y[idx]])
        # print(self.y[idx], xi.shape, yi.shape)
                
        return xi, yi

In [0]:
bs = 128
device = 'cuda'

In [0]:
train_dataset = BrainImages(out[0], out[1])
train_loader = DataLoader(train_dataset, batch_size=bs,
                        shuffle=False, num_workers=0)


val_dataset = BrainImages(out[2], out[3])
val_loader = DataLoader(val_dataset, batch_size=bs,
                        shuffle=False, num_workers=0)

In [0]:
d = next(iter(train_loader))

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.GRU(input_size=8, hidden_size=50,batch_first=True, num_layers=1)
        self.lin1 = nn.Linear(50,25)
        self.lin2 = nn.Linear(25, 1)
        self.sm = nn.Softmax()
        
    def forward(self, x):
        x = x.float()
        # print(x.shape)
        bss = x.size(0)
        output, hn = self.lstm(x)
        intr = hn[-1]
        oo = self.lin1(intr)
        oo = self.lin2(oo)
        return oo

In [0]:
def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0
    for data in iterator:
        optimizer.zero_grad()
        x = data[0].to(device)
        y = data[1].to(device).float()

        m_out = model(x)
        loss = criterion(m_out,y)

        epoch_loss += loss.mean().item()

        loss.backward()
        optimizer.step()
    print("Train Loss = ", epoch_loss)
    return model

def eval(model, iterator, criterion):
    model.eval()

    epoch_loss = 0
    total = 0
    correct = 0
    for data in iterator:
        x = data[0].to(device)
        y = data[1].to(device).float()

        with torch.no_grad():
            logits = model(x)

        loss = criterion(logits, y)
        epoch_loss+=loss.mean().item()

        total += x.size(0)

    # accuracy = correct/total
    print("Eval Loss = ", epoch_loss)
    # print("Eval Accuracy = ", accuracy)
    return epoch_loss

In [0]:
model = Net().to(device)
lr = 5e-5

In [0]:
criterion = nn.MSELoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [0]:
best_acc = np.inf
n_epochs = 500

In [28]:

for epoch in range(n_epochs):
    print("-"*10)
    print("Epoch = ", epoch+1)
    model = train(model, train_loader, optimizer, criterion)
    accuracy = eval(model, val_loader, criterion)

    # print(epoch, accuracy)
    if accuracy<best_acc:
        best_acc = accuracy
        torch.save(model, "model1.m")
        best_param = model.state_dict()

Eval Loss =  4271.301608085632
----------
Epoch =  470
Train Loss =  10464.504118204117
Eval Loss =  4270.848628997803
----------
Epoch =  471
Train Loss =  10460.403641700745
Eval Loss =  4270.405766010284
----------
Epoch =  472
Train Loss =  10456.300797462463
Eval Loss =  4269.972389221191
----------
Epoch =  473
Train Loss =  10452.20473241806
Eval Loss =  4269.551607131958
----------
Epoch =  474
Train Loss =  10448.10730624199
Eval Loss =  4269.139344215393
----------
Epoch =  475
Train Loss =  10444.012164354324
Eval Loss =  4268.739976406097
----------
Epoch =  476
Train Loss =  10439.916291475296
Eval Loss =  4268.3482394218445
----------
Epoch =  477
Train Loss =  10435.829503774643
Eval Loss =  4267.967306137085
----------
Epoch =  478
Train Loss =  10431.750391960144
Eval Loss =  4267.595892906189
----------
Epoch =  479
Train Loss =  10427.662987709045
Eval Loss =  4267.2304220199585
----------
Epoch =  480
Train Loss =  10423.591170310974
Eval Loss =  4266.875896453857
-

In [0]:
model.eval()

gt = []
tgt = []
for data in val_loader:
    x = data[0].to(device)
    y = data[1].to(device).float()

    with torch.no_grad():
        logits = model(x)
    gt.append(y.detach().cpu().numpy())
    tgt.append(logits.detach().cpu().numpy())

In [0]:
gt1 = np.vstack(gt)

In [0]:
tgt1 = np.vstack(tgt)

In [32]:
gt1.shape, tgt1.shape

((15013, 1), (15013, 1))

In [0]:
from sklearn.metrics import *

In [34]:
r2_score(gt1,tgt1)

0.8627830361369697